# Purpose

We want to investigate different ways of reading in a large CSV file when using Python. The goal is to compare the computational efficiency of different methods. We will use three methods, which are listed below:

* Pandas
* Dask
* Modin

# Import Libraries

In [1]:
import pandas as pd
import time
from dask import dataframe as dask_df
import modin.pandas as mpd

# The Data

The dataset used here is an artificial version of the iris dataset. To artificially increase the size of the file, the original dataset was concatenated with itself for many iterations. Thus, the new CSV file, called "big_iris", consists of many copies of the original iris dataset joined together. The size of the resulting CSV file is 2.9 GB.

# Comparing the different methods of reading in the CSV file

The code cells below read in the data using all 3 methods, and print out the time taken for each.

In [2]:
# Pandas 
s_time = time.time()
df1 = pd.read_csv("big_iris.csv")
e_time = time.time()
print("Pandas: ", (e_time-s_time), "seconds")

Pandas:  29.57462501525879 seconds


In [3]:
# Dask
s_time_dask = time.time()
df2 = dask_df.read_csv('big_iris.csv')
e_time_dask = time.time()
print("Read with dask: ", (e_time_dask-s_time_dask), "seconds")

Read with dask:  0.012249231338500977 seconds


In [4]:
# Modin
s_time_modin = time.time()
df3 = mpd.read_csv("big_iris.csv")
e_time_modin = time.time()
print("Modin: ", (e_time_modin-s_time_modin), "seconds")

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



Modin:  16.61344289779663 seconds


Our findings show that Dask is the most computationally efficient in this case, as it takes the least amount of time. Pandas takes the longest time to read in the data, and Modin is in between.

# Basic data validation

Here, we perform basic data validation on the columns of the data. We remove any special characters and white spaces that might be present.

In [5]:
# Drop first column from data 
df2 = df2.drop(columns=['Unnamed: 0'])

# Get all columns
df2.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [6]:
# Convert data type to string
df2 = df2.astype(str)

# Remove special characters
df2['Id'] = df2['Id'].str.replace('\W', '', regex=True)
df2['SepalLengthCm'] = df2['SepalLengthCm'].str.replace('\W', '', regex=True)
df2['SepalWidthCm'] = df2['SepalWidthCm'].str.replace('\W', '', regex=True)
df2['PetalLengthCm'] = df2['PetalLengthCm'].str.replace('\W', '', regex=True)
df2['PetalWidthCm'] = df2['PetalWidthCm'].str.replace('\W', '', regex=True)
df2['Species'] = df2['Species'].str.replace('\W', '', regex=True)

# Remove white spaces
df2['Id'].str.strip()
df2['SepalLengthCm'].str.strip()
df2['SepalWidthCm'].str.strip()
df2['PetalLengthCm'].str.strip()
df2['PetalWidthCm'].str.strip()
df2['Species'].str.strip()

Dask Series Structure:
npartitions=45
    object
       ...
     ...  
       ...
       ...
Name: Species, dtype: object
Dask Name: str-strip, 1035 tasks

# Writing testutil

In [7]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


# Write YAML file

In [8]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: big_iris
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Id
    - SepalLengthCm
    - SepalWidthCm
    - PetalLengthCm
    - PetalWidthCm
    - Species

Overwriting file.yaml


# Data Validation with YAML

In [9]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [10]:
# Inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'big_iris',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Id',
  'SepalLengthCm',
  'SepalWidthCm',
  'PetalLengthCm',
  'PetalWidthCm',
  'Species']}

In [11]:
# Normal reading process of the file
df_sample = pd.read_csv("big_iris.csv",delimiter=',')
df_sample = df_sample.drop(columns=['Unnamed: 0'])
df_sample.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [12]:
# Read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [13]:
# Validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [14]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['id', 'sepallengthcm', 'sepalwidthcm', 'petallengthcm', 'petalwidthcm',
       'species'],
      dtype='object')
columns of YAML are: ['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']


In [15]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation passed
col validation passed


# Writing the file

In [16]:
import csv
df.to_csv('my_file.txt', header=True, index=False, sep=config_data['outbound_delimiter'], mode='a')

# Summary of file

In [28]:
import os
print("The file has", df.shape[0], "rows")
print("The file has", df.shape[1], "columns")
file_size = os.path.getsize('my_file.txt')
file_size_gb = file_size * 10**-9
print("The file size is", file_size_gb, "gigabytes")

The file has 78643200 rows
The file has 6 columns
The file size is 7.930380483 gigabytes
